<img src="https://www3.um.edu.uy/logoum.jpg" width=300>
<h1 align="center">Thesis - IMF F1 </h1> 
<h2 align="center">Alejo Paullier</h2> 

https://lkpy.lenskit.org/en/stable/knn.html

In [1]:
from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, als
from lenskit.algorithms.als import ImplicitMF
import pandas as pd
import numpy as np
np.seterr(divide='ignore', invalid='ignore')

ratings = pd.read_csv('D:\\Escritorio\\UM\\Tesis\\ML 1M\\ratings.dat', sep='::',engine='python',
                      names=['user', 'item', 'rating', 'timestamp'])


def eval(aname, algo, train, test):
    fittable = util.clone(algo) # Object cloning means to create an exact copy of the original object.
    fittable = Recommender.adapt(fittable) 
    fittable.fit(train) # Entrenamos el algoritmo con el training dataset
    users = test.user.unique() # Devuelve un array con los users unicos en el testing dataset
    # now we run the recommender
    recs = batch.recommend(fittable, users, 100)
    # add the algorithm name for analyzability
    recs['Algorithm'] = aname
    return recs

In [2]:
def model_trainer(features,reg,weight):
    all_recs = []
    test_data = []
    
    features=int(features)
    imf = ImplicitMF(features=features, iterations=100, reg=reg, weight=weight) # define algorithm# define algorithm
    
    for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 5, xf.SampleFrac(0.2)):
        test_data.append(test) # save testing data
        all_recs.append(eval('IMF', imf, train, test))
     
    all_recs = pd.concat(all_recs, ignore_index=True)
    test_data = pd.concat(test_data, ignore_index=True)
    rla1 = topn.RecListAnalysis()
    rla2 = topn.RecListAnalysis()
    rla1.add_metric(topn.precision)
    rla2.add_metric(topn.recall)
    prec = rla1.compute(all_recs, test_data)
    rec = rla2.compute(all_recs, test_data)
    F1 = ((prec['precision'].values*rec['recall'].values*2)/(prec['precision'].values+rec['recall'].values))
    F1 = np.nan_to_num(F1)
    return [F1.mean(),prec['precision'].mean(),rec['recall'].mean()]

In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK
array=[]
def objective(params):
    features = params['features']
    reg = params['reg']
    weight=params['weight']
    metric = model_trainer(features,reg,weight)
    array.append([features,reg,weight,metric[0],metric[1],metric[2]])
    return {'loss': -metric[0], 'status': STATUS_OK }

space={'features': hp.uniform('features', 1, 100),
       'reg': hp.uniform('reg',0.01,0.4),
      'weight': hp.uniform('weight',0,100)}

best = fmin(objective, space, algo=tpe.suggest,max_evals=100)

  8%|███▋                                          | 8/100 [34:43<6:27:37, 252.80s/it, best loss: -0.16987010063242758]


KeyboardInterrupt: 

In [4]:
array #252.80

[[45.738897499164715,
  0.11144138463054225,
  55.560358470279795,
  0.13317742890102127,
  0.09195198675496576,
  0.4353340532760436],
 [75.04156655544207,
  0.3132030938613791,
  86.62476611568017,
  0.12607240214058738,
  0.08568543046357528,
  0.4186499294345618],
 [4.846657374120548,
  0.027091617947563373,
  90.96007838910684,
  0.10295550991886013,
  0.07453145695364125,
  0.3083489160352472],
 [2.4797789743566807,
  0.2983653340407876,
  47.19323826335733,
  0.10254997791933222,
  0.07547019867549558,
  0.29164674310504024],
 [20.212016382973125,
  0.1378154058505614,
  21.762992748458355,
  0.16987010063242758,
  0.12361423841059335,
  0.494254111805738],
 [57.82749319931563,
  0.17732865374690823,
  91.43302903956562,
  0.1203238981600427,
  0.08136754966887351,
  0.4096378885933068],
 [66.89639964564057,
  0.27400768284043353,
  41.59540095289893,
  0.1448757025535241,
  0.1007864238410584,
  0.45973925821860023],
 [17.84616486471195,
  0.2899721487777077,
  95.4895515918581

In [4]:
array #569.51

[[75.92096439904982,
  0.3245560491901431,
  20.206680139475242,
  0.16707177928640124,
  0.1190844370860904,
  0.5047411649827206],
 [12.567193867492175,
  0.1859938007867555,
  75.3857466053986,
  0.12229703574515781,
  0.08654470198675397,
  0.38351484372647937],
 [50.95899267382448,
  0.21241126531018356,
  20.73576168246145,
  0.1691414081085574,
  0.12115894039734869,
  0.507135935764667],
 [47.57276393786126,
  0.06635667530015929,
  37.57865007838601,
  0.1491469570792952,
  0.10463741721854165,
  0.46820300143389243],
 [70.31440701153285,
  0.3252593352328653,
  84.80964224852475,
  0.12473749296597801,
  0.08451158940397253,
  0.41798430766778827],
 [91.46667010221493,
  0.04874246143495871,
  30.24845354289254,
  0.15663614628614855,
  0.11004635761589242,
  0.48413988890777265],
 [78.81266158315925,
  0.28510087338604023,
  78.7905831681993,
  0.12873229212527293,
  0.08778642384105888,
  0.4224570042450235],
 [62.87259967627342,
  0.20652220985791936,
  9.085462291237755,


In [4]:
array #740.26

[[37.55423827148529,
  0.2434351355656911,
  73.71672755568743,
  0.12251474512218699,
  0.08386754966887347,
  0.4135146280164524],
 [18.40334673797654,
  0.07467711723599198,
  41.152287602422156,
  0.14922725300916634,
  0.10676986754966702,
  0.45337710620517246],
 [55.79258283037457,
  0.30688994744145576,
  78.44144251139141,
  0.12369853580593938,
  0.08397019867549607,
  0.41684676249033287],
 [47.98250326540918,
  0.23408754199469733,
  70.43080738318555,
  0.1253027470019288,
  0.08556125827814498,
  0.4215641481103843],
 [20.563229389929322,
  0.24347395910483624,
  78.95512982637365,
  0.12103704683749626,
  0.08425165562913822,
  0.3970633822001993],
 [72.08496786409941,
  0.23970284623533414,
  49.76893037292791,
  0.1398036042146009,
  0.09651490066225063,
  0.4504811299209238],
 [73.25802475397083,
  0.23812621849660703,
  75.17443881393443,
  0.1284664847555855,
  0.08775165562913803,
  0.4213969390982263],
 [65.56761989984457,
  0.1250419209679383,
  88.29125454275409

In [3]:
metric = pd.DataFrame(array)
metric.columns = ['features','reg','weight','F1','precision','recall']
metric.head(5)

,features,reg,weight,F1,precision,recall
0,75.920964,0.324556,20.206680,0.167072,0.119084,0.504741
1,12.567194,0.185994,75.385747,0.122297,0.086545,0.383515
2,50.958993,0.212411,20.735762,0.169141,0.121159,0.507136
3,47.572764,0.066357,37.578650,0.149147,0.104637,0.468203
4,70.314407,0.325259,84.809642,0.124737,0.084512,0.417984


In [4]:
metric.sort_values(by=['F1'],ascending=False).head(10)

,features,reg,weight,F1,precision,recall
66,84.505398,0.383922,0.538081,0.211535,0.158377,0.581117
89,52.194634,0.183721,2.122122,0.211012,0.157447,0.585043
70,81.594108,0.314689,0.202762,0.210599,0.158111,0.573956
21,85.491704,0.259203,0.924504,0.210569,0.157353,0.580831
23,84.136043,0.370140,1.186249,0.209786,0.156553,0.580266
61,65.233071,0.353213,2.018115,0.209718,0.156382,0.583232
72,93.678278,0.312021,0.337560,0.209116,0.156556,0.573546
24,97.951257,0.379180,0.722158,0.208574,0.155743,0.574443
29,92.763748,0.358182,0.173519,0.207716,0.155757,0.565629
43,97.378472,0.355231,0.238142,0.207257,0.155088,0.567972


In [5]:
metric.to_csv('metric_F1.csv', index=False)

In [1]:
array = [[75.92096439904982,
  0.3245560491901431,
  20.206680139475242,
  0.16707177928640124,
  0.1190844370860904,
  0.5047411649827206],
 [12.567193867492175,
  0.1859938007867555,
  75.3857466053986,
  0.12229703574515781,
  0.08654470198675397,
  0.38351484372647937],
 [50.95899267382448,
  0.21241126531018356,
  20.73576168246145,
  0.1691414081085574,
  0.12115894039734869,
  0.507135935764667],
 [47.57276393786126,
  0.06635667530015929,
  37.57865007838601,
  0.1491469570792952,
  0.10463741721854165,
  0.46820300143389243],
 [70.31440701153285,
  0.3252593352328653,
  84.80964224852475,
  0.12473749296597801,
  0.08451158940397253,
  0.41798430766778827],
 [91.46667010221493,
  0.04874246143495871,
  30.24845354289254,
  0.15663614628614855,
  0.11004635761589242,
  0.48413988890777265],
 [78.81266158315925,
  0.28510087338604023,
  78.7905831681993,
  0.12873229212527293,
  0.08778642384105888,
  0.4224570042450235],
 [62.87259967627342,
  0.20652220985791936,
  9.085462291237755,
  0.19020230648112654,
  0.13903973509933584,
  0.549711785769656],
 [93.246905039956,
  0.07978522540443554,
  78.64071171909241,
  0.13258703783342798,
  0.09089403973509867,
  0.4264861094858597],
 [58.14175169997161,
  0.11580277414398177,
  11.61165474710838,
  0.18436585109474476,
  0.13397185430463363,
  0.5391443160509297],
 [63.0805282457216,
  0.2333614002358988,
  60.48037252751172,
  0.13308900760457218,
  0.09131788079470123,
  0.43610222299103923],
 [56.12167468794812,
  0.1794792899383904,
  37.12443432108713,
  0.14865508873551572,
  0.10405298013244911,
  0.4684644417261503],
 [40.68002042304056,
  0.14717758875589615,
  65.20963097502171,
  0.12774659283894219,
  0.08775662251655536,
  0.4242280791087687],
 [25.10190442825404,
  0.32766512570166567,
  97.91900637088281,
  0.11058674835042236,
  0.07574337748344293,
  0.37829511959297035],
 [75.86669771567779,
  0.0626745207620898,
  61.58987671762066,
  0.13427184135459885,
  0.09222019867549558,
  0.4354015603407292],
 [33.89779518447058,
  0.17699912416330307,
  91.57662234679442,
  0.11483320477592546,
  0.07829966887417125,
  0.3926665309188255],
 [36.31705906063325,
  0.17344525740858155,
  73.09194623316937,
  0.12355657448267991,
  0.08478807947019781,
  0.41474311032070454],
 [9.368528413267628,
  0.16238290595970845,
  91.86161702925091,
  0.10968326155786398,
  0.07760264900662145,
  0.3458601991343342],
 [13.738587049216084,
  0.13434978008019927,
  50.22806169933227,
  0.13966885309436416,
  0.09991556291390558,
  0.4265550139219841],
 [66.2810425835869,
  0.17281911141191114,
  98.54849485261063,
  0.12052579481054149,
  0.08132615894039645,
  0.4084589761857242],
 [58.5303427536154,
  0.1132942237075102,
  3.3820425258158906,
  0.20572437981335956,
  0.1528327814569515,
  0.5754501645752813],
 [85.49170386694016,
  0.25920292020475943,
  0.9245042637244154,
  0.21056877467832463,
  0.15735264900662058,
  0.5808306686477999],
 [86.61599071430057,
  0.014950279036049716,
  4.5523766769974205,
  0.1984012824799779,
  0.1461837748344349,
  0.5634368843476486],
 [84.13604311236159,
  0.3701404623571988,
  1.1862491545771636,
  0.20978604236084022,
  0.1565529801324478,
  0.5802664969189641],
 [97.95125656484115,
  0.37918026915838116,
  0.7221584831526762,
  0.20857366741571348,
  0.1557433774834413,
  0.5744429935031807],
 [99.60280671790463,
  0.388722229202643,
  18.561981661776656,
  0.16935987476205913,
  0.12094701986754743,
  0.5078571183430127],
 [84.72103978274737,
  0.2584603691525704,
  13.504924249463938,
  0.17748657813535407,
  0.1278923841059582,
  0.5247321547087131],
 [83.06829954258978,
  0.3639286619831093,
  28.785308286837,
  0.15709550462401184,
  0.11058443708609085,
  0.48491102899933675],
 [74.74801293195873,
  0.27323228304324676,
  45.693848719371395,
  0.1431636495928163,
  0.09926490066225084,
  0.45626119350745836],
 [92.76374769305507,
  0.35818179894962665,
  0.17351878962592338,
  0.20771627882060445,
  0.1557566225165538,
  0.5656294375445559],
 [70.8459988367858,
  0.3109672877853828,
  22.99629175341021,
  0.16375915705278352,
  0.11631788079470003,
  0.4974492725329412],
 [99.81343864026714,
  0.23478097410410625,
  15.956110169438503,
  0.1731078788519276,
  0.12404470198675255,
  0.5169796299971162],
 [80.44368541501278,
  0.29873003927644604,
  24.697338331641912,
  0.16110444000819577,
  0.11401655629138881,
  0.4928264639110217],
 [47.49681913576487,
  0.3418221894576579,
  6.39293714573727,
  0.19896335522171307,
  0.14674337748344182,
  0.5651644629378751],
 [1.6113349321347101,
  0.2327828851831529,
  38.05578982468369,
  0.10710909962761292,
  0.0820331125827797,
  0.29494193977072036],
 [92.26692060861453,
  0.25294718101346964,
  50.52814916561715,
  0.14270542972681338,
  0.09886423841059447,
  0.4512231409750064],
 [87.40684359756087,
  0.39338660260725433,
  31.131626959718854,
  0.1553664440436444,
  0.10905629139072703,
  0.4814099008308196],
 [70.53092943401975,
  0.2918732329311877,
  7.941960931548015,
  0.1918862846247227,
  0.1404387417218523,
  0.5519452052851085],
 [45.42429548098718,
  0.33857316552647565,
  44.39517262675153,
  0.14198211316798892,
  0.09904635761589253,
  0.45221398449498057],
 [52.11197276993583,
  0.214715487019618,
  16.02174410946958,
  0.1767187492830072,
  0.12752152317880583,
  0.5233879620520598],
 [78.39330774716075,
  0.19801126372147268,
  26.031697563268875,
  0.16027733950537754,
  0.1131970198675479,
  0.49231495150894256],
 [24.157376689494825,
  0.3126395007486715,
  10.054298563250295,
  0.18904008850546009,
  0.1395231788079449,
  0.5363429061667803],
 [65.10642915666651,
  0.2731715416495233,
  33.925318782926254,
  0.1518674560742891,
  0.10639238410595851,
  0.47519952950462224],
 [97.37847238880116,
  0.3552309796043308,
  0.23814236292741453,
  0.20725693199945075,
  0.1550877483443681,
  0.5679723918620265],
 [90.16968940904714,
  0.375718386731993,
  19.7991582039463,
  0.16805938089877917,
  0.11965894039734862,
  0.5074194972157785],
 [53.89807419082557,
  0.09147727711570526,
  67.50866331517616,
  0.12817993671800942,
  0.08751986754966821,
  0.426563105407033],
 [72.62311268344195,
  0.39733968762204225,
  41.80608729179137,
  0.14541302753512883,
  0.1010033112582766,
  0.4617153276918999],
 [60.16652084830451,
  0.21964273676092627,
  53.45705336877253,
  0.13687613996980838,
  0.09450827814569422,
  0.44156279066270626],
 [79.80032376859342,
  0.25882759554064955,
  55.69903446389638,
  0.1379546254798587,
  0.09499337748344294,
  0.44456807662995196],
 [67.81471319673936,
  0.32351444021768866,
  85.39280308416662,
  0.12445006162485071,
  0.08440066225165513,
  0.4162241989634684],
 [95.49372688043195,
  0.03638160007868113,
  11.629269115089137,
  0.1800238997040702,
  0.13017384105960048,
  0.5289582453967089],
 [39.68809844608056,
  0.19624135856767935,
  71.64167336848092,
  0.12416186102104215,
  0.0851307947019861,
  0.41566572993478723],
 [87.17674585984417,
  0.15438069420750505,
  3.8881381090064737,
  0.20058427190239397,
  0.14815066225165316,
  0.5674366926565563],
 [27.162382422070856,
  0.13116394776371984,
  34.98415279330746,
  0.15512248004859194,
  0.11052980132450133,
  0.4732471529969578],
 [61.26539301349405,
  0.2828768708584101,
  14.990888440837988,
  0.17775903892917644,
  0.1282417218543023,
  0.5266655762714805],
 [75.71598234503344,
  0.3040784810326439,
  21.295415732280922,
  0.16556803771130787,
  0.11771357615893872,
  0.5026186739138081],
 [82.19114334743544,
  0.33640824831773186,
  27.53897998090197,
  0.1589526509996308,
  0.11212417218542862,
  0.4893140913921865],
 [31.396168298878294,
  0.3730524012733455,
  58.54921838142715,
  0.1328226447067992,
  0.09236920529801215,
  0.432035066456685],
 [55.042191538839276,
  0.2483001263730504,
  79.06083706239662,
  0.12310627373870199,
  0.08371026490066151,
  0.41412549119800346],
 [90.0328242033506,
  0.09526215751977282,
  40.140419822532756,
  0.14827020510445058,
  0.10353973509933652,
  0.4626933097381408],
 [45.078382813979424,
  0.27211863893331517,
  31.38552176937703,
  0.15562717358693512,
  0.11000496688741518,
  0.4810481392634257],
 [65.23307092581521,
  0.35321323452751957,
  2.018114667502452,
  0.2097184014339253,
  0.15638245033112397,
  0.5832317398588295],
 [16.82529379718487,
  0.22469663641131774,
  7.297831089396887,
  0.18703223545841063,
  0.13945860927152065,
  0.5200929422134707],
 [95.76157491687442,
  0.32276964802376273,
  18.577894929515544,
  0.1694214651850676,
  0.12090231788079234,
  0.5101796487808252],
 [68.23184914653855,
  0.18805664560410862,
  46.09119302356806,
  0.1419317223609349,
  0.09824834437085964,
  0.454606631381738],
 [63.939753166723435,
  0.35022633504883127,
  3.1433271705666996,
  0.20721512972236347,
  0.15391556291390507,
  0.5795055536004723],
 [84.50539782901272,
  0.3839222038809152,
  0.5380806734308483,
  0.21153547646255527,
  0.15837748344370609,
  0.5811172283804603],
 [84.88119133012934,
  0.3871191739939831,
  11.530653377693689,
  0.18138284080343295,
  0.1313741721854285,
  0.5323007784248566],
 [77.46052163814096,
  0.3700655615246925,
  5.8223266971545105,
  0.19559111961430972,
  0.1438990066225144,
  0.5573257426201738],
 [72.85765093600659,
  0.24343198525279097,
  23.525621124188508,
  0.1631707269362304,
  0.11581291390728302,
  0.4950098031265766],
 [81.59410753031366,
  0.3146892402932664,
  0.20276206462930169,
  0.21059928929892738,
  0.15811092715231564,
  0.5739560770439092],
 [88.81775771361637,
  0.2854053359165275,
  9.277869477803222,
  0.18552809432811268,
  0.13499668874171972,
  0.5399747783779663],
 [93.67827750356301,
  0.31202140532463296,
  0.33755983799043443,
  0.20911599038194487,
  0.15655629139072647,
  0.5735464306299891],
 [81.58047041198535,
  0.2612584515253219,
  12.926189675349196,
  0.17871678064408314,
  0.12904139072847495,
  0.528173631888607],
 [58.849586787668244,
  0.3322657529527681,
  17.38442231336845,
  0.1742455283326265,
  0.1251705298013222,
  0.5201268820139952],
 [49.54439066057112,
  0.3186802135684251,
  21.4884145606063,
  0.1683922403560132,
  0.1205016556291369,
  0.5078435208398494],
 [98.55642165319938,
  0.2978154444692033,
  5.291978448375292,
  0.19399067271042583,
  0.14254470198675293,
  0.5542572710465239],
 [73.53237243484945,
  0.2670422932642242,
  14.819495945042851,
  0.1760899692434232,
  0.126769867549667,
  0.5230296524327046],
 [77.14119547037231,
  0.20454707138519462,
  8.48932005160672,
  0.18878262510717195,
  0.13776986754966666,
  0.5458054861012995],
[37.55423827148529,
  0.2434351355656911,
  73.71672755568743,
  0.12251474512218699,
  0.08386754966887347,
  0.4135146280164524],
 [18.40334673797654,
  0.07467711723599198,
  41.152287602422156,
  0.14922725300916634,
  0.10676986754966702,
  0.45337710620517246],
 [55.79258283037457,
  0.30688994744145576,
  78.44144251139141,
  0.12369853580593938,
  0.08397019867549607,
  0.41684676249033287],
 [47.98250326540918,
  0.23408754199469733,
  70.43080738318555,
  0.1253027470019288,
  0.08556125827814498,
  0.4215641481103843],
 [20.563229389929322,
  0.24347395910483624,
  78.95512982637365,
  0.12103704683749626,
  0.08425165562913822,
  0.3970633822001993],
 [72.08496786409941,
  0.23970284623533414,
  49.76893037292791,
  0.1398036042146009,
  0.09651490066225063,
  0.4504811299209238],
 [73.25802475397083,
  0.23812621849660703,
  75.17443881393443,
  0.1284664847555855,
  0.08775165562913803,
  0.4213969390982263],
 [65.56761989984457,
  0.1250419209679383,
  88.29125454275409,
  0.1232508404368496,
  0.08336920529801287,
  0.4137274193014593],
 [47.46722724414297,
  0.38631382655032126,
  74.36443803753028,
  0.12354271522279005,
  0.08417384105960178,
  0.41789411802750026],
 [89.96510333026997,
  0.3904014958261058,
  33.96008057947253,
  0.15280754420495296,
  0.10698675496688599,
  0.47474390700412056],
 [52.194633966008276,
  0.18372116203114078,
  2.122122376529345,
  0.2110120491991216,
  0.1574470198675476,
  0.5850433913234784],
 [37.19319901133312,
  0.2318769733844595,
  37.053241580994666,
  0.15047528186334214,
  0.10616059602648883,
  0.46914591852344983],
 [64.03219370870735,
  0.07617268349168198,
  52.9438402926378,
  0.13732499762864772,
  0.09473344370860834,
  0.44264161026258],
 [62.78420298855572,
  0.3496945002915359,
  75.30063742575841,
  0.12574720816077992,
  0.08560264900662155,
  0.42000668969072663],
 [39.110594073337765,
  0.11531719780983518,
  94.84966242196961,
  0.11427987403308176,
  0.07739403973509866,
  0.3939420791595462],
 [62.74871610632231,
  0.08875506863936179,
  57.47984486304377,
  0.13394348331992698,
  0.09210596026489967,
  0.4371006382964464],
        [45.738897499164715,
  0.11144138463054225,
  55.560358470279795,
  0.13317742890102127,
  0.09195198675496576,
  0.4353340532760436],
 [75.04156655544207,
  0.3132030938613791,
  86.62476611568017,
  0.12607240214058738,
  0.08568543046357528,
  0.4186499294345618],
 [4.846657374120548,
  0.027091617947563373,
  90.96007838910684,
  0.10295550991886013,
  0.07453145695364125,
  0.3083489160352472],
 [2.4797789743566807,
  0.2983653340407876,
  47.19323826335733,
  0.10254997791933222,
  0.07547019867549558,
  0.29164674310504024],
 [20.212016382973125,
  0.1378154058505614,
  21.762992748458355,
  0.16987010063242758,
  0.12361423841059335,
  0.494254111805738],
 [57.82749319931563,
  0.17732865374690823,
  91.43302903956562,
  0.1203238981600427,
  0.08136754966887351,
  0.4096378885933068],
 [66.89639964564057,
  0.27400768284043353,
  41.59540095289893,
  0.1448757025535241,
  0.1007864238410584,
  0.45973925821860023],
 [17.84616486471195,
  0.2899721487777077,
  95.48955159185815,
  0.1116821950482671,
  0.07754304635761511,
  0.3713039064616846]]

In [4]:
metric = pd.read_csv('C:\\Users\\Alejo\\Tesis\\1M\\Implicit Matrix Factorization\\results\\metric_F1.csv')
metric

,features,reg,weight,F1,precision,recall
0,75.920964,0.324556,20.206680,0.167072,0.119084,0.504741
1,12.567194,0.185994,75.385747,0.122297,0.086545,0.383515
2,50.958993,0.212411,20.735762,0.169141,0.121159,0.507136
3,47.572764,0.066357,37.578650,0.149147,0.104637,0.468203
4,70.314407,0.325259,84.809642,0.124737,0.084512,0.417984
...,...,...,...,...,...,...
95,45.738898,0.111441,55.560358,0.133177,0.091952,0.435334
96,75.041567,0.313203,86.624766,0.126072,0.085685,0.418650
97,4.846657,0.027092,90.960078,0.102956,0.074531,0.308349
98,2.479779,0.298365,47.193238,0.102550,0.075470,0.291647


In [7]:
%matplotlib qt
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
colormap = cm.hot
x = metric['reg']
y = metric['features'] 
z = metric['weight']
c = metric['F1']

fig = plt.figure()

ax = fig.add_subplot(111, projection='3d')
img = ax.scatter(x, y, z, c=c, cmap=plt.jet())
ax.set_xlabel('reg')
ax.set_ylabel('features')
ax.set_zlabel('weight')
fig.colorbar(img)
plt.show()